## Benchmarking with Argo Worfklows & Vegeta

In this notebook we will dive into how you can run bench marking with batch processing with Argo Workflows, Seldon Core and Vegeta.

Dependencies:

* Seldon core installed as per the docs with Istio as an ingress 
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Note: If running with KIND you need to make sure do follow [these steps](https://github.com/argoproj/argo/issues/2376#issuecomment-595593237) as workaround to the `/.../docker.sock` known issue.


### Install Argo Workflows
You can follow the instructions from the official [Argo Workflows Documentation](https://github.com/argoproj/argo#quickstart).

Download the right CLi for your environment following the documentation (https://github.com/argoproj/argo-workflows/releases/tag/v3.0.8)

You also need to make sure that argo has permissions to create seldon deployments - for this you can just create a default-admin rolebinding as follows:

In [302]:
!kubectl create namespace argo || echo "namespace already created"
!kubectl apply -n argo -f https://raw.githubusercontent.com/argoproj/argo/v3.1.0/manifests/install.yaml
!kubectl rollout status -n argo deployment/argo-server
!kubectl rollout status -n argo deployment/workflow-controller

Error from server (AlreadyExists): namespaces "argo" already exists
namespace already created
customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io unchanged
serviceaccount/argo unchanged
serviceaccount/argo-server unchanged
role.rbac.authorization.k8s.io/argo-role unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-admin unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-edit unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-view unchanged
clusterrole.rbac.authorization.k8s.io/argo-cluster-role unchanged
clusterrole.rbac.authorization.k8s.io/argo-server-cluster-role unchanged
rolebi

Set up the RBAC so the argo workflow is able to create seldon deployments.

In [5]:
!kubectl create rolebinding default-admin --clusterrole=admin --serviceaccount=default:default
print("Successfully installed admin role-binding")

rolebinding.rbac.authorization.k8s.io/default-admin created


### Create Benchmark Argo Workflow

In order to create a benchmark, we created a simple argo workflow template so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 1 replicas and 4 cpus (with 100 max workers) to send requests.

In [490]:
print("Running argo workflow job")
!helm template seldon-benchmark-workflow helm-charts/seldon-benchmark-workflow/ \
    --set workflow.name=seldon-benchmark-process \
    --set workflow.parallelism=3 \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas="1|2" \
    --set seldonDeployment.serverWorkers="2|5|10" \
    --set seldonDeployment.serverThreads=1 \
    --set seldonDeployment.modelUri="gs://seldon-models/sklearn/iris" \
    --set seldonDeployment.server="SKLEARN_SERVER" \
    --set seldonDeployment.apiType="rest|grpc" \
    --set seldonDeployment.requests.cpu="500Mi|1000Mi|2000Mi" \
    --set seldonDeployment.requests.limits="2000Mi" \
    --set seldonDeployment.disableOrchestrator="true|false" \
    --set benchmark.cpu="5" \
    --set benchmark.concurrency="1" \
    --set benchmark.duration="30s" \
    --set benchmark.rate=0 \
    --set benchmark.data='\{"data": {"ndarray": [[0\,1\,2\,3]]\}\}' \
    | argo submit -

Name:                seldon-benchmark-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Sat Jun 26 18:39:08 +0100 (now)


In [515]:
print("Running argo workflows:")
!argo list

Running argo workflows:
NAME   STATUS   AGE   DURATION   PRIORITY


In [ ]:
!argo logs -f seldon-benchmark-process

In [ ]:
!argo get seldon-benchmark-process

## Process the results

We can now print the results in a consumable format.

In [494]:
def get_results(results, print_results=True):
    final = {}
    # For GHZ / grpc
    if "average" in results:
        final["mean"] = results["average"] / 1e6
        if results.get("latencyDistribution", False):
            final["50th"] = results["latencyDistribution"][-5]["latency"] / 1e6
            final["90th"] = results["latencyDistribution"][-3]["latency"] / 1e6
            final["95th"] = results["latencyDistribution"][-2]["latency"] / 1e6
            final["99th"] = results["latencyDistribution"][-1]["latency"] / 1e6
        final["throughputAchieved"] = results["rps"]
        final["success"] = results["statusCodeDistribution"].get("OK", 0)
        final["errors"] = sum(results["statusCodeDistribution"].values()) - final["success"]
    # For vegeta / rest
    else:
        final["mean"] = results["latencies"]["mean"] / 1e6
        final["50th"] = results["latencies"]["50th"] / 1e6
        final["90th"] = results["latencies"]["90th"] / 1e6
        final["95th"] = results["latencies"]["95th"] / 1e6
        final["99th"] = results["latencies"]["99th"] / 1e6
        final["throughputAchieved"] = results["throughput"]
        final["success"] = results["status_codes"].get("200", 0)
        final["errors"] = sum(results["status_codes"].values()) - final["success"]
    for k in results["params"].keys():
        final[k] = results["params"][k]
    if print_results:
        print("-----")
        print("ParamNames:", results["params"].keys())
        print("ParamNames:", results["params"].values())
        print("\tLatencies:")
        print("\t\tmean:", final["mean"], "ms")
        print("\t\t50th:", final["50th"], "ms")
        print("\t\t90th:", final["90th"], "ms")
        print("\t\t95th:", final["95th"], "ms")
        print("\t\t99th:", final["99th"], "ms")
        print("")
        print("\tRate:", str(final["throughputAchieved"]) + "/s")
        print("\tSuccess:", final["success"])
        print("\tErrors:", final["errors"])
    return final

In [ ]:
wf_logs = !argo logs --no-color seldon-benchmark-process 
wf_bench = [json.loads(":".join(w.split(":")[1:])) for w in wf_logs if "latenc" in w]
print(wf_bench)

In [ ]:
results = []
for w in wf_bench:
    # Prints the results in a consumable format
    results.append(get_results(w))

In [514]:
!argo delete seldon-benchmark-process || echo "Argo workflow already deleted or not exists"

Workflow 'seldon-benchmark-process' deleted


In [ ]:
!kubectl delete -n argo -f https://raw.githubusercontent.com/argoproj/argo/v3.1.0/manifests/install.yaml

## Deeper Analysis
Now that we have all the parameters, we can do a deeper analysis

In [497]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df.head()

,mean,50th,90th,95th,99th,throughputAchieved,success,errors,name,replicas,...,apiType,requestsCpu,requestsMemory,limitsCpu,limitsMemory,benchmarkCpu,concurrency,duration,rate,disableOrchestrator
0,6.828045,6.390520,9.264792,10.765109,14.086176,146.276304,4389,0,sklearn-1,1,...,rest,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
1,5.401355,5.254559,6.584599,7.153629,8.984036,184.916561,5549,0,sklearn-5,1,...,rest,2000Mi,100Mi,50m,1000Mi,5,1,30s,0,false
2,5.291129,5.183541,6.444586,7.040763,9.199069,188.787112,5665,0,sklearn-4,1,...,rest,2000Mi,100Mi,50m,1000Mi,5,1,30s,0,true
3,4.646862,4.383044,5.952588,6.775633,9.886973,215.054059,6453,0,sklearn-15,1,...,rest,1000Mi,100Mi,50m,1000Mi,5,1,30s,0,false
4,6.568330,6.142206,8.891155,10.257702,13.721939,152.068454,4563,0,sklearn-0,1,...,rest,500Mi,100Mi,50m,1000Mi,5,1,30s,0,true
